In [16]:
import re
import pandas as pd
from pathlib import Path

# Emoji handling
import emoji

# Sastrawi (Bahasa Indonesia)
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Inisialisasi stemmer & stopwords
stemmer = StemmerFactory().create_stemmer()
stopwords_id = set(StopWordRemoverFactory().get_stop_words())

# (Opsional) tambahkan stopwords domain KAI
stopwords_domain = {
    "kai","kaiaccess","access","kereta","krl","min","admin",
    "tolong","mohon","terima","kasih","terimakasih"
}
stopwords_all = stopwords_id.union(stopwords_domain)

pd.set_option("display.max_colwidth", 200)


In [17]:
# === Step 2: Load File ===
INPUT_FILE = "/Users/ahmadzaki/Downloads/Skripsi Zaki/CodeSkripsi/01.Cleaning/data/FINALBGT_kai_matched_by_textlabel.csv"

use_dtypes = {"id_str":"string","user_id_str":"string","conversation_id_str":"string"}
df = pd.read_csv(INPUT_FILE, dtype=use_dtypes, low_memory=False)

# Pastikan kolom teks ada
assert "full_text" in df.columns, "Kolom 'full_text' tidak ditemukan pada CSV."

print("Loaded:", INPUT_FILE)
print("Rows:", len(df))
df[["created_at","id_str","user_id_str","conversation_id_str","full_text","label"]].head(5)

Loaded: /Users/ahmadzaki/Downloads/Skripsi Zaki/CodeSkripsi/01.Cleaning/data/FINALBGT_kai_matched_by_textlabel.csv
Rows: 1434


,created_at,id_str,user_id_str,conversation_id_str,full_text,label
0,Fri Dec 30 09:57:34 +0000 2022,1608764220451213313,230609460,1608764220451213313,udah beli tiket di aplikasi kaiaccess dan saat pembelian dan pembayaran mengikutsertakan asuransi perjalanan Sompo Insurance Indonesia tetapi belum ada imel masuk dari asuransi utk polisnya bagaim...,NEGATIF
1,Fri Dec 30 01:11:02 +0000 2022,1608631716184489986,1158718283870138368,1608631716184489986,@KAI121 kenapa gabisa pake ovo buat pembayaran kaiaccess min ?,NEGATIF
2,Thu Dec 29 00:47:15 +0000 2022,1608263340564058116,1250731212471074816,1608263340564058116,@KAI121 min saya mau ubah no. Hp yang ada di kaiaccess. Mohon bantuannya min,NETRAL
3,Wed Dec 28 22:55:35 +0000 2022,1608235238366535680,120352418,1608227080503955458,Waktunya menikmati pemandangan sepanjang perjalanan ini ditemani segelas teh premium nikmat dan segar. Pesannya tadi lewat #KAIAccess dan bayarnya pakai gopay. Mudah dan praktis. @KAI121 https://t...,POSITIF
4,Wed Dec 28 10:38:04 +0000 2022,1608049638111850496,967360502652223489,1608049638111850496,@KAI121 kenapa KAIAccess dari kemaren susah di acces loading terus,NEGATIF


In [18]:
# === Step 3: Case Folding ===
work = df.copy()
work["text_casefold"] = work["full_text"].fillna("").str.lower()

work[["full_text","text_casefold"]].head(10)

,full_text,text_casefold
0,udah beli tiket di aplikasi kaiaccess dan saat pembelian dan pembayaran mengikutsertakan asuransi perjalanan Sompo Insurance Indonesia tetapi belum ada imel masuk dari asuransi utk polisnya bagaim...,udah beli tiket di aplikasi kaiaccess dan saat pembelian dan pembayaran mengikutsertakan asuransi perjalanan sompo insurance indonesia tetapi belum ada imel masuk dari asuransi utk polisnya bagaim...
1,@KAI121 kenapa gabisa pake ovo buat pembayaran kaiaccess min ?,@kai121 kenapa gabisa pake ovo buat pembayaran kaiaccess min ?
2,@KAI121 min saya mau ubah no. Hp yang ada di kaiaccess. Mohon bantuannya min,@kai121 min saya mau ubah no. hp yang ada di kaiaccess. mohon bantuannya min
3,Waktunya menikmati pemandangan sepanjang perjalanan ini ditemani segelas teh premium nikmat dan segar. Pesannya tadi lewat #KAIAccess dan bayarnya pakai gopay. Mudah dan praktis. @KAI121 https://t...,waktunya menikmati pemandangan sepanjang perjalanan ini ditemani segelas teh premium nikmat dan segar. pesannya tadi lewat #kaiaccess dan bayarnya pakai gopay. mudah dan praktis. @kai121 https://t...
4,@KAI121 kenapa KAIAccess dari kemaren susah di acces loading terus,@kai121 kenapa kaiaccess dari kemaren susah di acces loading terus
5,@KAI121 halo saya abis beli tiket kereta di aplikasi kaiaccess udah saya bayar saldo saya kepotong tapi tiketnya belum kebeli sedangkan batas waktu habis dan tiket ga terbit,@kai121 halo saya abis beli tiket kereta di aplikasi kaiaccess udah saya bayar saldo saya kepotong tapi tiketnya belum kebeli sedangkan batas waktu habis dan tiket ga terbit
6,@KAI121 min aplikasi kaiaccess lagi ngadat kah? berat banget skg ngeload datanya. giliran udah tampil sering banget gabisa diproses munculnya server bekerja terlalu keras. Kalian ada masalah apa? ...,@kai121 min aplikasi kaiaccess lagi ngadat kah? berat banget skg ngeload datanya. giliran udah tampil sering banget gabisa diproses munculnya server bekerja terlalu keras. kalian ada masalah apa? ...
7,ada yang tau jam buka loket stasiun sidoarjo beli tiket kereta bukanya jam berapa? mau beli tiket yg keberangkatan pagi soalnya takut belum buka beli nde kaiaccess ga punya saldo samsek :( Sdf!,ada yang tau jam buka loket stasiun sidoarjo beli tiket kereta bukanya jam berapa? mau beli tiket yg keberangkatan pagi soalnya takut belum buka beli nde kaiaccess ga punya saldo samsek :( sdf!
8,Halo @KAI121 apps nya lagi under constructions apa jaringan sinyal @myXLCare yg kurang baik di Cilacap? Bolak-balik-bolak matiin mode pesawat masuk App #KaiAccess ga bisa² mau pesen tiket aja mash...,halo @kai121 apps nya lagi under constructions apa jaringan sinyal @myxlcare yg kurang baik di cilacap? bolak-balik-bolak matiin mode pesawat masuk app #kaiaccess ga bisa² mau pesen tiket aja mash...
9,@KAI121 Hallo @KAI121 kenapaa app KAIaccess tidak bisa digunakan yaa?,@kai121 hallo @kai121 kenapaa app kaiaccess tidak bisa digunakan yaa?


In [19]:
# === Step 4: Cleaning ===
# def clean_text(s: str) -> str:
#     s = str(s)
#     s = re.sub(r"http\S+|www\.\S+", " ", s)     # URL
#     s = re.sub(r"@\w+", " ", s)                 # mention
#     s = re.sub(r"#", " ", s)                    # hapus simbol '#' (biarkan kata hashtag)
#     s = re.sub(r"[^\w\s]", " ", s)              # hapus tanda baca (non huruf/angka/underscore)
#     s = re.sub(r"\s+", " ", s).strip()          # rapikan spasi
#     return s

# === Step 4: Cleaning ===
def clean_text(s: str) -> str:
    s = str(s)
    s = re.sub(r"http\S+|www\.\S+", " ", s)       # hapus URL
    s = re.sub(r"@\w+", " ", s)                   # hapus mention
    s = re.sub(r"#", " ", s)                      # hapus simbol '#'
    s = re.sub(r"\d+", " NUM ", s)                # ganti angka dengan token NUM
    s = re.sub(r"(.)\1{2,}", r"\1", s)            # hilangkan huruf berulang lebih dari 2x (e.g., "bagusss" → "bagus")
    s = re.sub(r"[^\w\s]", " ", s)                # hapus tanda baca
    s = re.sub(r"_", " ", s)                      # hapus underscore
    s = re.sub(r"\s+", " ", s).strip()            # rapikan spasi
    return s

work["text_clean"] = work["text_casefold"].apply(clean_text)

work[["text_casefold","text_clean"]].head(10)

,text_casefold,text_clean
0,udah beli tiket di aplikasi kaiaccess dan saat pembelian dan pembayaran mengikutsertakan asuransi perjalanan sompo insurance indonesia tetapi belum ada imel masuk dari asuransi utk polisnya bagaim...,udah beli tiket di aplikasi kaiaccess dan saat pembelian dan pembayaran mengikutsertakan asuransi perjalanan sompo insurance indonesia tetapi belum ada imel masuk dari asuransi utk polisnya bagaim...
1,@kai121 kenapa gabisa pake ovo buat pembayaran kaiaccess min ?,kenapa gabisa pake ovo buat pembayaran kaiaccess min
2,@kai121 min saya mau ubah no. hp yang ada di kaiaccess. mohon bantuannya min,min saya mau ubah no hp yang ada di kaiaccess mohon bantuannya min
3,waktunya menikmati pemandangan sepanjang perjalanan ini ditemani segelas teh premium nikmat dan segar. pesannya tadi lewat #kaiaccess dan bayarnya pakai gopay. mudah dan praktis. @kai121 https://t...,waktunya menikmati pemandangan sepanjang perjalanan ini ditemani segelas teh premium nikmat dan segar pesannya tadi lewat kaiaccess dan bayarnya pakai gopay mudah dan praktis
4,@kai121 kenapa kaiaccess dari kemaren susah di acces loading terus,kenapa kaiaccess dari kemaren susah di acces loading terus
5,@kai121 halo saya abis beli tiket kereta di aplikasi kaiaccess udah saya bayar saldo saya kepotong tapi tiketnya belum kebeli sedangkan batas waktu habis dan tiket ga terbit,halo saya abis beli tiket kereta di aplikasi kaiaccess udah saya bayar saldo saya kepotong tapi tiketnya belum kebeli sedangkan batas waktu habis dan tiket ga terbit
6,@kai121 min aplikasi kaiaccess lagi ngadat kah? berat banget skg ngeload datanya. giliran udah tampil sering banget gabisa diproses munculnya server bekerja terlalu keras. kalian ada masalah apa? ...,min aplikasi kaiaccess lagi ngadat kah berat banget skg ngeload datanya giliran udah tampil sering banget gabisa diproses munculnya server bekerja terlalu keras kalian ada masalah apa tolong seger...
7,ada yang tau jam buka loket stasiun sidoarjo beli tiket kereta bukanya jam berapa? mau beli tiket yg keberangkatan pagi soalnya takut belum buka beli nde kaiaccess ga punya saldo samsek :( sdf!,ada yang tau jam buka loket stasiun sidoarjo beli tiket kereta bukanya jam berapa mau beli tiket yg keberangkatan pagi soalnya takut belum buka beli nde kaiaccess ga punya saldo samsek sdf
8,halo @kai121 apps nya lagi under constructions apa jaringan sinyal @myxlcare yg kurang baik di cilacap? bolak-balik-bolak matiin mode pesawat masuk app #kaiaccess ga bisa² mau pesen tiket aja mash...,halo apps nya lagi under constructions apa jaringan sinyal yg kurang baik di cilacap bolak balik bolak matiin mode pesawat masuk app kaiaccess ga bisa² mau pesen tiket aja masha alloh sabar sabar
9,@kai121 hallo @kai121 kenapaa app kaiaccess tidak bisa digunakan yaa?,hallo kenapaa app kaiaccess tidak bisa digunakan yaa


In [21]:
import re

# === Step 4b: Normalization ===

def load_slang_dicts():
    # Kamus 1: dari GitHub
    url = 'https://raw.githubusercontent.com/ezaaputra/Sentiment-Analysis-Using-BERT/main/kamus_alay.csv'
    df1 = pd.read_csv(url, names=["slang", "formal"], encoding="utf-8")
    
    # Gabungkan & buang duplikat
    slang_dict = pd.concat([df1]).drop_duplicates(subset="slang", keep="last")
    slang_dict = dict(zip(slang_dict["slang"], slang_dict["formal"]))
    return slang_dict

slang_dict = load_slang_dicts()

def normalize_text(text):
    text = str(text).lower()
    words = text.split()
    normalized = [slang_dict.get(w, w) for w in words]
    text = " ".join(normalized)
    text = re.sub(r'(.)\1{2,}', r'\1', text)  # hapus huruf berulang
    return text

work["text_normalized"] = work["text_clean"].apply(normalize_text)

work[["text_casefold","text_normalized"]].head(10)


,text_casefold,text_normalized
0,udah beli tiket di aplikasi kaiaccess dan saat pembelian dan pembayaran mengikutsertakan asuransi perjalanan sompo insurance indonesia tetapi belum ada imel masuk dari asuransi utk polisnya bagaim...,sudah beli tiket di aplikasi kaiaccess dan saat pembelian dan pembayaran mengikutsertakan asuransi perjalanan sompo insurance indonesia tetapi belum ada imel masuk dari asuransi untuk polisnya bag...
1,@kai121 kenapa gabisa pake ovo buat pembayaran kaiaccess min ?,kenapa enggak bisa pakai ovo buat pembayaran kaiaccess min
2,@kai121 min saya mau ubah no. hp yang ada di kaiaccess. mohon bantuannya min,min saya mau ubah no hp yang ada di kaiaccess mohon bantuannya min
3,waktunya menikmati pemandangan sepanjang perjalanan ini ditemani segelas teh premium nikmat dan segar. pesannya tadi lewat #kaiaccess dan bayarnya pakai gopay. mudah dan praktis. @kai121 https://t...,waktunya menikmati pemandangan sepanjang perjalanan ini ditemani segelas teh premium nikmat dan segar pesannya tadi lewat kaiaccess dan bayarnya pakai gopay mudah dan praktis
4,@kai121 kenapa kaiaccess dari kemaren susah di acces loading terus,kenapa kaiaccess dari kemarin susah di acces loading terus
5,@kai121 halo saya abis beli tiket kereta di aplikasi kaiaccess udah saya bayar saldo saya kepotong tapi tiketnya belum kebeli sedangkan batas waktu habis dan tiket ga terbit,halo saya habis beli tiket kereta di aplikasi kaiaccess sudah saya bayar saldo saya kepotong tapi tiketnya belum kebeli sedangkan batas waktu habis dan tiket enggak terbit
6,@kai121 min aplikasi kaiaccess lagi ngadat kah? berat banget skg ngeload datanya. giliran udah tampil sering banget gabisa diproses munculnya server bekerja terlalu keras. kalian ada masalah apa? ...,min aplikasi kaiaccess lagi ngadat kah berat banget sekarang ngeload datanya giliran sudah tampil sering banget enggak bisa diproses munculnya server bekerja terlalu keras kalian ada masalah apa t...
7,ada yang tau jam buka loket stasiun sidoarjo beli tiket kereta bukanya jam berapa? mau beli tiket yg keberangkatan pagi soalnya takut belum buka beli nde kaiaccess ga punya saldo samsek :( sdf!,ada yang tau jam buka loket stasiun sidoarjo beli tiket kereta bukanya jam berapa mau beli tiket yang keberangkatan pagi soalnya takut belum buka beli nde kaiaccess enggak punya saldo samsek sdf
8,halo @kai121 apps nya lagi under constructions apa jaringan sinyal @myxlcare yg kurang baik di cilacap? bolak-balik-bolak matiin mode pesawat masuk app #kaiaccess ga bisa² mau pesen tiket aja mash...,halo apps nya lagi under constructions apa jaringan sinyal yang kurang baik di cilacap bolak balik bolak matiin mode pesawat masuk app kaiaccess enggak bisa² mau pesen tiket saja masha allah sabar...
9,@kai121 hallo @kai121 kenapaa app kaiaccess tidak bisa digunakan yaa?,halo kenapa app kaiaccess tidak bisa digunakan ya


In [22]:
# === Step 5: Tokenizing ===
def tokenize(s: str):
    return str(s).split()

work["tokens"] = work["text_normalized"].apply(tokenize)

work[["text_normalized","tokens"]].head(10)


,text_normalized,tokens
0,sudah beli tiket di aplikasi kaiaccess dan saat pembelian dan pembayaran mengikutsertakan asuransi perjalanan sompo insurance indonesia tetapi belum ada imel masuk dari asuransi untuk polisnya bag...,"[sudah, beli, tiket, di, aplikasi, kaiaccess, dan, saat, pembelian, dan, pembayaran, mengikutsertakan, asuransi, perjalanan, sompo, insurance, indonesia, tetapi, belum, ada, imel, masuk, dari, asu..."
1,kenapa enggak bisa pakai ovo buat pembayaran kaiaccess min,"[kenapa, enggak, bisa, pakai, ovo, buat, pembayaran, kaiaccess, min]"
2,min saya mau ubah no hp yang ada di kaiaccess mohon bantuannya min,"[min, saya, mau, ubah, no, hp, yang, ada, di, kaiaccess, mohon, bantuannya, min]"
3,waktunya menikmati pemandangan sepanjang perjalanan ini ditemani segelas teh premium nikmat dan segar pesannya tadi lewat kaiaccess dan bayarnya pakai gopay mudah dan praktis,"[waktunya, menikmati, pemandangan, sepanjang, perjalanan, ini, ditemani, segelas, teh, premium, nikmat, dan, segar, pesannya, tadi, lewat, kaiaccess, dan, bayarnya, pakai, gopay, mudah, dan, praktis]"
4,kenapa kaiaccess dari kemarin susah di acces loading terus,"[kenapa, kaiaccess, dari, kemarin, susah, di, acces, loading, terus]"
5,halo saya habis beli tiket kereta di aplikasi kaiaccess sudah saya bayar saldo saya kepotong tapi tiketnya belum kebeli sedangkan batas waktu habis dan tiket enggak terbit,"[halo, saya, habis, beli, tiket, kereta, di, aplikasi, kaiaccess, sudah, saya, bayar, saldo, saya, kepotong, tapi, tiketnya, belum, kebeli, sedangkan, batas, waktu, habis, dan, tiket, enggak, terbit]"
6,min aplikasi kaiaccess lagi ngadat kah berat banget sekarang ngeload datanya giliran sudah tampil sering banget enggak bisa diproses munculnya server bekerja terlalu keras kalian ada masalah apa t...,"[min, aplikasi, kaiaccess, lagi, ngadat, kah, berat, banget, sekarang, ngeload, datanya, giliran, sudah, tampil, sering, banget, enggak, bisa, diproses, munculnya, server, bekerja, terlalu, keras,..."
7,ada yang tau jam buka loket stasiun sidoarjo beli tiket kereta bukanya jam berapa mau beli tiket yang keberangkatan pagi soalnya takut belum buka beli nde kaiaccess enggak punya saldo samsek sdf,"[ada, yang, tau, jam, buka, loket, stasiun, sidoarjo, beli, tiket, kereta, bukanya, jam, berapa, mau, beli, tiket, yang, keberangkatan, pagi, soalnya, takut, belum, buka, beli, nde, kaiaccess, eng..."
8,halo apps nya lagi under constructions apa jaringan sinyal yang kurang baik di cilacap bolak balik bolak matiin mode pesawat masuk app kaiaccess enggak bisa² mau pesen tiket saja masha allah sabar...,"[halo, apps, nya, lagi, under, constructions, apa, jaringan, sinyal, yang, kurang, baik, di, cilacap, bolak, balik, bolak, matiin, mode, pesawat, masuk, app, kaiaccess, enggak, bisa², mau, pesen, ..."
9,halo kenapa app kaiaccess tidak bisa digunakan ya,"[halo, kenapa, app, kaiaccess, tidak, bisa, digunakan, ya]"


In [23]:
# === Step 6: Stopword Removal ===
work["tokens_nostop"] = work["tokens"].apply(
    lambda toks: [t for t in toks if t not in stopwords_all]
)

work[["tokens","tokens_nostop"]].head(10)


,tokens,tokens_nostop
0,"[sudah, beli, tiket, di, aplikasi, kaiaccess, dan, saat, pembelian, dan, pembayaran, mengikutsertakan, asuransi, perjalanan, sompo, insurance, indonesia, tetapi, belum, ada, imel, masuk, dari, asu...","[beli, tiket, aplikasi, pembelian, pembayaran, mengikutsertakan, asuransi, perjalanan, sompo, insurance, indonesia, imel, masuk, asuransi, polisnya, bagaimana]"
1,"[kenapa, enggak, bisa, pakai, ovo, buat, pembayaran, kaiaccess, min]","[enggak, pakai, ovo, buat, pembayaran]"
2,"[min, saya, mau, ubah, no, hp, yang, ada, di, kaiaccess, mohon, bantuannya, min]","[mau, ubah, no, hp, bantuannya]"
3,"[waktunya, menikmati, pemandangan, sepanjang, perjalanan, ini, ditemani, segelas, teh, premium, nikmat, dan, segar, pesannya, tadi, lewat, kaiaccess, dan, bayarnya, pakai, gopay, mudah, dan, praktis]","[waktunya, menikmati, pemandangan, sepanjang, perjalanan, ditemani, segelas, teh, premium, nikmat, segar, pesannya, tadi, lewat, bayarnya, pakai, gopay, mudah, praktis]"
4,"[kenapa, kaiaccess, dari, kemarin, susah, di, acces, loading, terus]","[kemarin, susah, acces, loading, terus]"
5,"[halo, saya, habis, beli, tiket, kereta, di, aplikasi, kaiaccess, sudah, saya, bayar, saldo, saya, kepotong, tapi, tiketnya, belum, kebeli, sedangkan, batas, waktu, habis, dan, tiket, enggak, terbit]","[halo, habis, beli, tiket, aplikasi, bayar, saldo, kepotong, tiketnya, kebeli, batas, waktu, habis, tiket, enggak, terbit]"
6,"[min, aplikasi, kaiaccess, lagi, ngadat, kah, berat, banget, sekarang, ngeload, datanya, giliran, sudah, tampil, sering, banget, enggak, bisa, diproses, munculnya, server, bekerja, terlalu, keras,...","[aplikasi, ngadat, berat, banget, sekarang, ngeload, datanya, giliran, tampil, sering, banget, enggak, diproses, munculnya, server, bekerja, terlalu, keras, kalian, masalah, apa, segera, cari, ban..."
7,"[ada, yang, tau, jam, buka, loket, stasiun, sidoarjo, beli, tiket, kereta, bukanya, jam, berapa, mau, beli, tiket, yang, keberangkatan, pagi, soalnya, takut, belum, buka, beli, nde, kaiaccess, eng...","[tau, jam, buka, loket, stasiun, sidoarjo, beli, tiket, bukanya, jam, berapa, mau, beli, tiket, keberangkatan, pagi, soalnya, takut, buka, beli, nde, enggak, punya, saldo, samsek, sdf]"
8,"[halo, apps, nya, lagi, under, constructions, apa, jaringan, sinyal, yang, kurang, baik, di, cilacap, bolak, balik, bolak, matiin, mode, pesawat, masuk, app, kaiaccess, enggak, bisa², mau, pesen, ...","[halo, apps, nya, under, constructions, apa, jaringan, sinyal, kurang, baik, cilacap, bolak, balik, bolak, matiin, mode, pesawat, masuk, app, enggak, bisa², mau, pesen, tiket, masha, allah, sabar,..."
9,"[halo, kenapa, app, kaiaccess, tidak, bisa, digunakan, ya]","[halo, app, digunakan]"


In [24]:
# === Step 7: Stemming ===
def stem_tokens(tokens):
    return [stemmer.stem(t) for t in tokens]

work["tokens_stem"] = work["tokens_nostop"].apply(stem_tokens)
work["text_stemmed"] = work["tokens_stem"].apply(lambda ts: " ".join(ts))

work[["tokens_nostop","tokens_stem","text_stemmed"]].head(10)

,tokens_nostop,tokens_stem,text_stemmed
0,"[beli, tiket, aplikasi, pembelian, pembayaran, mengikutsertakan, asuransi, perjalanan, sompo, insurance, indonesia, imel, masuk, asuransi, polisnya, bagaimana]","[beli, tiket, aplikasi, beli, bayar, mengikutsertakan, asuransi, jalan, sompo, insurance, indonesia, imel, masuk, asuransi, polis, bagaimana]",beli tiket aplikasi beli bayar mengikutsertakan asuransi jalan sompo insurance indonesia imel masuk asuransi polis bagaimana
1,"[enggak, pakai, ovo, buat, pembayaran]","[enggak, pakai, ovo, buat, bayar]",enggak pakai ovo buat bayar
2,"[mau, ubah, no, hp, bantuannya]","[mau, ubah, no, hp, bantu]",mau ubah no hp bantu
3,"[waktunya, menikmati, pemandangan, sepanjang, perjalanan, ditemani, segelas, teh, premium, nikmat, segar, pesannya, tadi, lewat, bayarnya, pakai, gopay, mudah, praktis]","[waktu, nikmat, pandang, panjang, jalan, tani, gelas, teh, premium, nikmat, segar, pesan, tadi, lewat, bayar, pakai, gopay, mudah, praktis]",waktu nikmat pandang panjang jalan tani gelas teh premium nikmat segar pesan tadi lewat bayar pakai gopay mudah praktis
4,"[kemarin, susah, acces, loading, terus]","[kemarin, susah, acces, loading, terus]",kemarin susah acces loading terus
5,"[halo, habis, beli, tiket, aplikasi, bayar, saldo, kepotong, tiketnya, kebeli, batas, waktu, habis, tiket, enggak, terbit]","[halo, habis, beli, tiket, aplikasi, bayar, saldo, potong, tiket, kebel, batas, waktu, habis, tiket, enggak, terbit]",halo habis beli tiket aplikasi bayar saldo potong tiket kebel batas waktu habis tiket enggak terbit
6,"[aplikasi, ngadat, berat, banget, sekarang, ngeload, datanya, giliran, tampil, sering, banget, enggak, diproses, munculnya, server, bekerja, terlalu, keras, kalian, masalah, apa, segera, cari, ban...","[aplikasi, ngadat, berat, banget, sekarang, ngeload, data, gilir, tampil, sering, banget, enggak, proses, muncul, server, kerja, terlalu, keras, kalian, masalah, apa, segera, cari, bantu, biar, se...",aplikasi ngadat berat banget sekarang ngeload data gilir tampil sering banget enggak proses muncul server kerja terlalu keras kalian masalah apa segera cari bantu biar segera maksimal layan
7,"[tau, jam, buka, loket, stasiun, sidoarjo, beli, tiket, bukanya, jam, berapa, mau, beli, tiket, keberangkatan, pagi, soalnya, takut, buka, beli, nde, enggak, punya, saldo, samsek, sdf]","[tau, jam, buka, loket, stasiun, sidoarjo, beli, tiket, buka, jam, berapa, mau, beli, tiket, berangkat, pagi, soal, takut, buka, beli, nde, enggak, punya, saldo, samsek, sdf]",tau jam buka loket stasiun sidoarjo beli tiket buka jam berapa mau beli tiket berangkat pagi soal takut buka beli nde enggak punya saldo samsek sdf
8,"[halo, apps, nya, under, constructions, apa, jaringan, sinyal, kurang, baik, cilacap, bolak, balik, bolak, matiin, mode, pesawat, masuk, app, enggak, bisa², mau, pesen, tiket, masha, allah, sabar,...","[halo, apps, nya, under, constructions, apa, jaring, sinyal, kurang, baik, cilacap, bolak, balik, bolak, matiin, mode, pesawat, masuk, app, enggak, bisa, mau, sen, tiket, masha, allah, sabar, sabar]",halo apps nya under constructions apa jaring sinyal kurang baik cilacap bolak balik bolak matiin mode pesawat masuk app enggak bisa mau sen tiket masha allah sabar sabar
9,"[halo, app, digunakan]","[halo, app, guna]",halo app guna


In [25]:
# === Step 8: Emoji Handling (Drop vs Map) ===

def emoji_drop(s: str) -> str:
    return emoji.replace_emoji(str(s), replace="")

def emoji_map(s: str) -> str:
    # contoh hasil: "bagus 😊" -> "bagus :smiling_face_with_smiling_eyes:"
    return emoji.demojize(str(s), language="en")

work["emoji_drop_from_raw"] = work["full_text"].apply(emoji_drop)
work["emoji_map_from_raw"]  = work["full_text"].apply(emoji_map)

# Tampilkan contoh baris yang kemungkinan mengandung emoji
def has_emoji(text: str) -> bool:
    return emoji.emoji_count(str(text)) > 0

sample_emoji = work[work["full_text"].apply(has_emoji)].head(15)
sample_emoji[["full_text","emoji_drop_from_raw","emoji_map_from_raw"]]


,full_text,emoji_drop_from_raw,emoji_map_from_raw
61,Min @CommuterLine .. usulan saya kapan hari malah tambah parah nih server KAIAccess.. pesan lgsg TRK-NT gak bisa estafet baru bisa itupun nunggu sejam. Ini mslhnya apa IT SYSTEM? SERVER? Biaya MTC...,Min @CommuterLine .. usulan saya kapan hari malah tambah parah nih server KAIAccess.. pesan lgsg TRK-NT gak bisa estafet baru bisa itupun nunggu sejam. Ini mslhnya apa IT SYSTEM? SERVER? Biaya MTC...,Min @CommuterLine .. usulan saya kapan hari malah tambah parah nih server KAIAccess.. pesan lgsg TRK-NT gak bisa estafet baru bisa itupun nunggu sejam. Ini mslhnya apa IT SYSTEM? SERVER? Biaya MTC...
1272,Yuk segera beli tiket kereta api mu di Aplikasi Access by KAI bayar dengan MotionPay‼️ *s&amp;k berlaku #LiburanPraktis #MotionPay,Yuk segera beli tiket kereta api mu di Aplikasi Access by KAI bayar dengan MotionPay *s&amp;k berlaku #LiburanPraktis #MotionPay,Yuk segera beli tiket kereta api mu di Aplikasi Access by KAI bayar dengan MotionPay:double_exclamation_mark: *s&amp;k berlaku #LiburanPraktis #MotionPay


In [26]:
def emoji_drop(s: str) -> str:
    """Hapus emoji sepenuhnya dari teks."""
    return emoji.replace_emoji(str(s), replace="")

def emoji_map(s: str) -> str:
    """Ubah emoji menjadi teks deskriptif, mis. 😭 -> :loudly_crying_face:"""
    return emoji.demojize(str(s), language="en")

# Terapkan ke kolom hasil stemming
work["emoji_drop_from_stemmed"] = work["text_stemmed"].apply(emoji_drop)
work["emoji_map_from_stemmed"]  = work["text_stemmed"].apply(emoji_map)

# Fungsi bantu: cek apakah teks mengandung emoji
def has_emoji(text: str) -> bool:
    return emoji.emoji_count(str(text)) > 0

# Ambil sampel baris yang mengandung emoji setelah stemming
sample_emoji_stemmed = work[work["text_stemmed"].apply(has_emoji)].head(15)
sample_emoji_stemmed[["text_stemmed", "emoji_drop_from_stemmed", "emoji_map_from_stemmed"]]


,text_stemmed,emoji_drop_from_stemmed,emoji_map_from_stemmed


In [27]:
# === Step 9: Save Stepwise Result (optional) ===
OUT_STEPWISE = "data/preprocessed_A_stepwise_with_normalisasi.csv"
cols = [
    "id_str","created_at","user_id_str","conversation_id_str",
    "full_text",
    "text_casefold","text_clean","text_normalized","tokens","tokens_nostop",
    "tokens_stem","text_stemmed","label"
]
cols = [c for c in cols if c in work.columns]
work.to_csv(OUT_STEPWISE, index=False, encoding="utf-8-sig")
print("Saved:", OUT_STEPWISE, "| rows:", len(work))

Saved: data/preprocessed_A_stepwise_with_normalisasi.csv | rows: 1434


In [15]:
# === Step 9 (Modified): Save Simplified Preprocessed File ===

# Tentukan file output
OUT_SIMPLIFIED = "data/preprocessed_A_final_for_model.csv"

# Pilih kolom yang dibutuhkan
cols_to_export = ["id_str", "created_at", "full_text", "text_stemmed", "label"]
cols_to_export = [c for c in cols_to_export if c in work.columns]

# Ekspor ke CSV
work[cols_to_export].to_csv(OUT_SIMPLIFIED, index=False, encoding="utf-8-sig")

print(f"✅ File preprocessed final disimpan: {OUT_SIMPLIFIED}")
print(f"Total baris: {len(work)} | Total kolom: {len(cols_to_export)}")
print("Kolom yang disimpan:", cols_to_export)


✅ File preprocessed final disimpan: data/preprocessed_A_final_for_model.csv
Total baris: 1434 | Total kolom: 5
Kolom yang disimpan: ['id_str', 'created_at', 'full_text', 'text_stemmed', 'label']


Pedoman Label:
- pos (positif): pujian, pengalaman baik, kepuasan, terima kasih (yang bermakna positif).
- neg (negatif): keluhan, error, gagal, marah, kecewa, sarkas yang jelas negatif.
- neu (netral): informasi, tanya murni (tanpa emosi), klarifikasi, permintaan bantuan tanpa opini.

Catatan:
- Fokus pada SENTIMEN terhadap layanan KAI Access (aplikasi/fitur/proses).
- Kalau kalimat campuran, nilai sentimen dominan.
- Abaikan balasan resmi KAI (sudah kita hapus sebelumnya).
- Isi kolom LABEL dengan: pos | neg | neu (huruf kecil, konsisten).
- Boleh isi catatan di kolom NOTE jika ragu (nanti bisa direview).
